# Check created networks and plot example network

After generating networks, it is important to check that they have the properties we expect. This notebook will read in a set of graph models and plot properties such as edge weight distribution.

In [ ]:
## Import packages

using Statistics
using Pkg
using LinearAlgebra
using StatsBase
using Random
using JLD
using Plots
using Distributions
using JSON
using MAT
Pkg.add("ColorSchemes")
using ColorSchemes
Pkg.add("PyCall")
using PyCall
nx = pyimport("networkx")
np = pyimport("numpy")

include("helper_functions.jl")

## Set parameters

The following will extract parameters from the designated configuration file. After running this block, you should see a list of graph files found that will be analyzed.

In [ ]:
# Read from config file
config_file = "config101220.json"
config = read_config("$(homedir())/configs/$(config_file)")

const NNODES = config["NNODES"]
const MAXDIM = config["MAXDIM"]
const NREPS = config["NREPS"]
const DATE_STRING = config["DATE_STRING"]
const SAVETAIL = config["SAVETAIL_graphInfo"]
const read_dir = "$(homedir())/$(config["read_dir_graphs"])/$(NNODES)nodes"


# Locate and print graph files
graph_files = filter(x->occursin(DATE_STRING,x), readdir(read_dir))

println("Located the following graph files:")
for graph_file in graph_files
    println(graph_file)
end

# Extract just the model names
model_names = [split(graph_file,"_")[1] for graph_file in graph_files]



## Generate plots

The following will create one .pdf for each network model (each graph file found). The .pdf file by default will contain:

- Heatmap of last network adjacency matrix
- Strength distribution of the last network
- Edge weight distribution of the last network
- Averaged adjacency matrix over replicates
- Distribution of node strength using all replicates
- Distribution of edge weights using all replicates
- Distribution of the average clustering coefficient
- Distribution of modularity

***Note*** The modularity calculations can take a significant amount of time. Consider commenting out these plots and calculations if they are not useful.

In [ ]:
# Set styles for all plots. Though the resulting plots have funky colors/fonts, this makes it easier to 
# edit later in illustrator.
default(titlefont = (20, "times"), legendfontsize = 18, guidefont = (18, "times", :darkgreen), tickfont = (12, :orange, "times"), guide = "x", framestyle = :zerolines, yminorgrid = true)

# Loop over graph files and create a .pdf for each.
for graph_file in graph_files
    
    # Extract weighted graph array
    graph_dict = load("$(read_dir)/$(graph_file)")
    weighted_graph_array = graph_dict["weighted_graph_array"]
    nReps = size(weighted_graph_array)[3]

    # Find last graph and calculate average graph
    G_end = weighted_graph_array[:,:,nReps]
    G_avg = dropdims(mean(weighted_graph_array, dims=3), dims=3)

    # Heatmap of last graph
    p1a = heatmap(G_end,yflip = true, aspect_ratio=:equal, grid = false, c = :gist_yarg, axis=false, xlims=[1,70],
        framestyle=:box)
    title!("Last graph")
    ylabel!("Nodes")
    xlabel!("Nodes")
    
    # Strength distributions - last graph
    p1b = histogram([sum(G_end, dims=1)...], legend = false, c=:gray, alpha = 0.3)
    title!("Strength distribution last graph")
    ylabel!("Frequency")
    xlabel!("Node strength")


    # Edge weight distributions - last graph
    p1c = histogram(triu_elements(G_end,1), legend = false, c=:gray, alpha = 0.3)
    title!("Last graph edge weight distribution")
    ylabel!("Frequency")
    xlabel!("Edge weight")
    
    # Heatmap - average graph
    p1d = heatmap(G_avg,yflip = true, aspect_ratio=:equal, grid=false, c = :gist_yarg, xlims=[1,70], framestyle=:box)
    title!("Average graph")
    ylabel!("Nodes")
    xlabel!("Nodes")
    
    # Strength distribution - average graph
    p1e = histogram([sum(weighted_graph_array, dims=1)...], legend = false, c=:gray, alpha = 0.3)
    title!("Strength distribution all graphs")
    ylabel!("Frequency")
    xlabel!("Node strength")

    # Edge weight distribution - average graph
    p1f = histogram([weighted_graph_array...], legend = false, c=:gray, alpha = 0.3)
    title!("All graphs edge weight distribution")
    ylabel!("Frequency")
    xlabel!("Edge weight")


    # Graph metrics
    clustering = zeros(1,nReps)
    modularity = zeros(1,nReps)


    for rep in 1:nReps

        g = nx.from_numpy_matrix(weighted_graph_array[:,:,rep], parallel_edges=false)
        clustering[1, rep] = nx.average_clustering(g, weight = "weight")
        modularity[1, rep] = nx.algorithms.community.modularity(g, nx.algorithms.community.label_propagation_communities(g))

    end

    # Clustering distribution 
    p1g = histogram([clustering...], legend=false, c=:gray, alpha = 0.3)
    title!("Average clustering (all graphs)")
    xlabel!("Avg. clustering")
    ylabel!("Frequency")

    # Modularity distribution
    p1h = histogram([modularity...], legend=false, c=:gray, alpha = 0.3)
    title!("Modularity (all graphs)")
    xlabel!("Modularity")
    ylabel!("Frequency")


    # Add all subplots to main plot and write to pdf
    pi1 = plot(p1a, p1b, p1c, p1d, p1e, p1f, p1g, p1h, layout = (8, 1), size=(400,2000))
    savefig("../figures/$(replace(graph_file, "_graphs.jld" => ""))_$(SAVETAIL).pdf")

end

